# Machine learning Part 1: Supervised learning

## Exercise 1: Training and testing a model

### Load data

We will use data from the Autism Brain Imaging Data Exchange (ABIDE) dataset. 
The data has already been downloaded and preprocessed into two TSV tables:
- [`participants_nbsub-100.tsv`](../../data/participants_nbsub-100.tsv)
    - Phenotypic information: participant age, sex, scan site, diagnosis, etc.
- [`abide_nbsub-100_atlas-ho_meas-correlation_relmat.tsv`](../../data/abide_nbsub-100_atlas-ho_meas-correlation_relmat.tsv)
    - Flattened functional connectivity matrixes

The script used to create these files can be found [here](../../data/build_datasets.py).

Let's load and display the phenotypic data:

In [ ]:
import pandas as pd

data_dir = "../../data"

# phenotypic data (including diagnosis group DX_GROUP)
pheno_data_tsv = f"{data_dir}/participants_nbsub-100.tsv"
pheno_df = pd.read_csv(pheno_data_tsv, sep="\t", index_col=0)
pheno_df

And now the brain data:

In [ ]:
# brain data: functional connectivity matrices (flattened)
brain_data_tsv = f"{data_dir}/abide_nbsub-100_atlas-ho_meas-correlation_relmat.tsv"
brain_df = pd.read_csv(brain_data_tsv, sep="\t", index_col=0)
brain_df

### Try predicting diagnosis

Now we will try to predict diagnosis using the functional connectivity measures.

First, let's write a helper function that takes as input a matrix `X`, a vector `y` and an `sklearn` model `model`. The function should:
1. Split the data into train and test sets
2. Fit the model on the train data
3. Compute and print model performance on the train and test sets
4. Return the fitted classifier model

Write/modify the code in the cell below where it says `TODO`:

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


def train_and_test_model(
    model,
    X: pd.DataFrame,
    y: pd.Series,
    test_subset_fraction=0.2,
    shuffle=True,
    do_stratify=True,
    random_state=123,
):
    """Train and test a scikit-learn model.

    Parameters
    ----------
    model :
        The scikit-learn model to be trained and tested
    X : pd.DataFrame
        Input features
    y : pd.Series
        Output labels
    test_subset_fraction : float, optional
        Fraction of the dataset to use for testing, by default 0.2
    shuffle : bool, optional
        Whether to shuffle the data before splitting, by default True
    random_state : int, optional
        Seed to force the shuffle to be the same each time, by default 123

    Returns
    -------
        The fitted model
    """
    if do_stratify:
        # ensure similar distribution of classes in train and test sets
        stratify = y
    else:
        stratify = None

    # TODO divide the data into train/test sets
    # Hint: use sklearn's train_test_split function here
    #       make sure to use test_subset_fraction, shuffle, stratify, and random_state
    X_train, X_test, y_train, y_test = ...

    # TODO fit the model

    # TODO compute the train and test accuracies
    acc_train = ...
    acc_test = ...

    # print accuracies
    print(f"Train accuracy: {acc_train:.3f}")
    print(f"Test accuracy:  {acc_test:.3f}")

    return model

Now use this function to test a logistic regression model ([`sklearn.linear_model.LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html))

In [ ]:
from sklearn.linear_model import LogisticRegression

X = brain_df
y = pheno_df["DX_GROUP"]  # diagnosis

# TODO define your model
model = ...

# TODO call train_and_test_model() with the appropriate arguments

#### Questions

What do the train and test accuracies tell us? 

What can we try to improve test performance?

### Try predicting scan site instead of diagnosis

How about we try predicting another variable instead: scan site

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

X = brain_df  # same input data
y = pheno_df["SITE_ID"]  # scan site
y = pd.Series(LabelEncoder().fit_transform(y))  # encode sites as integers

# TODO: define your model
model = ...

# TODO call train_and_test_model() with the appropriate arguments

#### Questions

What do these performance metrics tell us?

How do we know if this is a good model? What is the chance performance?

## Exercise 2: Model selection and cross-validation

We can get a more stable estimate of a model's generalization ability by using cross-validation instead of a single test set.

Update/complete the `cross_validate_model` function below to use the [`StratifiedKFold` class](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) to create cross-validation splits for the dataset, then train and test models separately for each split.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold


def cross_validate_model(
    model,
    X: pd.DataFrame,
    y: pd.Series,
    n_splits=5,
    shuffle=True,
    random_state=123,
):
    """Train and test a scikit-learn model with cross-validation.

    Parameters
    ----------
    model :
        The scikit-learn model to be trained and tested
    X : pd.DataFrame
        Input features
    y : pd.Series
        Output labels
    n_splits : int, optional
        Number of folds for cross-validation, by default 5
    shuffle : bool, optional
        Whether to shuffle the data before splitting, by default True
    random_state : int, optional
        Seed to force the shuffle to be the same each time, by default 123

    Returns
    -------
    list
        The fitted models
    """
    # create lists to store results for each fold
    fitted_models = []
    accs_train = []
    accs_test = []

    # TODO create a StratifiedKFold cross-validator object
    # make sure to use the n_splits, shuffle, and random_state variables
    cv = ...

    # TODO iterate over the folds
    # Hint: look at the documentation of StratifiedKFold
    for train_index, test_index in ...:

        # TODO get the train and test sets using the indices
        # hint: X and y and pandas objects, you can use .iloc[] on them
        X_train = ...
        X_test = ...
        y_train = ...
        y_test = ...

        # get a fresh (unfitted) model
        model = clone(model)

        # TODO fit the model

        # TODO get the train and test accuracies
        acc_train = ...
        acc_test = ...

        # append results
        fitted_models.append(model)
        accs_train.append(acc_train)
        accs_test.append(acc_test)

    # report the mean accuracies
    accs_train = np.array(accs_train)
    accs_test = np.array(accs_test)
    print(f"Train accuracy: {accs_train.mean():.3f} ± {accs_train.std():.3f}")
    print(f"Test accuracy:  {accs_test.mean():.3f} ± {accs_test.std():.3f}")

    return fitted_models

We can now use our new cross-validation function to predict scan site again, but let's switch it up a bit and use Support Vector Machines (implemented as `sklearn.svm.SVC`) instead of logistic regression.

In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

X = brain_df
y = pheno_df["SITE_ID"]  # scanning site
y = pd.Series(LabelEncoder().fit_transform(y))  # encode as integers

# TODO: define your model
model = ...

# TODO call cross_validate_model() with the appropriate parameters

You may have noticed that the models we are using have arguments that can be specified when defining/instantiating the model. For example `SVC` has a regularization **hyperparameter** called `C`. Changing the value of `C` may change the model's performance, but doing so in a trial-and-error way leads to implicit data leakage (can you think of why?).

Sklearn's `GridSearchCV` class allows us to specify a hyperparameter grid to search over when fitting the data. The test data is not used during that process. Let's use `GridSearchCV` to test several values of `C`: `[0.01, 0.1, 1, 10, 100]`.

Note: what `GridSearchCV` does is called **inner** cross-validation, which should not be confused with the cross-validation done in `cross_validate_model`. Inner cross-validation is used to choose the best set of hyperparameters for a model, while outer cross-validation is used for getting a measure of model generalizability on new data.

In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

X = brain_df
y = pheno_df["SITE_ID"]  # scanning site
y = pd.Series(LabelEncoder().fit_transform(y))  # encode as integers

# TODO: define your model
model = ...

# TODO call cross_validate_model() with the appropriate parameters
models = ...

Print the `C` that was chosen for each CV fold. Is it the same for all folds?

Hint: What does the `cross_validate_model` function return?

Hint: Look at the [`GridSearchCV` documentation page](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to see how to access the best estimator for each fold.

Hint: The documentation for the [`SVC` class](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) may also be useful.

In [ ]:
# your code here

### Question

How should we go about comparing different models (e.g. `LogisticRegression` vs `SVC`?)